In [1]:
import numpy as np
import librosa
from scipy import stats, signal
import pandas as pd
from scipy.fft import fft
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import euclidean_distances

In [3]:
def extract_enhanced_mfcc_features(mfcc_matrix):
    """
    Extract enhanced features from MFCC coefficients matrix, specifically designed
    for music and voice classification.
    
    Parameters:
    mfcc_matrix: numpy array of shape (20, n_timestamps)
        The MFCC coefficients matrix
    
    Returns:
    dict: Dictionary containing extracted features
    """
    features = {}
    
    # 1. Basic Statistical Features (from previous code)
    for i in range(20):
        coeff = mfcc_matrix[i, :]
        
        # Calculate first-order deltas (velocity)
        delta = librosa.feature.delta(coeff.reshape(1, -1), width=19)
        
        # Calculate second-order deltas (acceleration)
        delta_delta = librosa.feature.delta(delta, order=2, width=19)
        
        # Basic statistics for original coefficients
        features[f'mfcc{i}_mean'] = np.mean(coeff)
        features[f'mfcc{i}_std'] = np.std(coeff)
        features[f'mfcc{i}_median'] = np.median(coeff)
        
        # Statistics for deltas (velocity features)
        features[f'mfcc{i}_delta_mean'] = np.mean(delta)
        features[f'mfcc{i}_delta_std'] = np.std(delta)
        features[f'mfcc{i}_delta_max'] = np.max(np.abs(delta))
        
        # Statistics for delta-deltas (acceleration features)
        features[f'mfcc{i}_delta2_mean'] = np.mean(delta_delta)
        features[f'mfcc{i}_delta2_std'] = np.std(delta_delta)
        features[f'mfcc{i}_delta2_max'] = np.max(np.abs(delta_delta))
        
        # Temporal variation features
        features[f'mfcc{i}_delta_zero_crossings'] = np.sum(np.diff(np.signbit(delta)))
        features[f'mfcc{i}_delta2_zero_crossings'] = np.sum(np.diff(np.signbit(delta_delta)))
        
        # Energy-related features
        features[f'mfcc{i}_delta_energy'] = np.sum(delta ** 2)
        features[f'mfcc{i}_delta2_energy'] = np.sum(delta_delta ** 2)
        
        # Ratio features
        features[f'mfcc{i}_delta_energy_ratio'] = features[f'mfcc{i}_delta_energy'] / np.sum(coeff ** 2)
        features[f'mfcc{i}_delta2_energy_ratio'] = features[f'mfcc{i}_delta2_energy'] / features[f'mfcc{i}_delta_energy']

    # 2. Rhythm-based Features
    for i in range(20):
        coeff = mfcc_matrix[i, :]
        
        # Detect peaks in the coefficient
        peaks, _ = signal.find_peaks(coeff)
        if len(peaks) > 1:
            # Average distance between peaks (rhythm indicator)
            features[f'mfcc{i}_peak_distance_mean'] = np.mean(np.diff(peaks))
            features[f'mfcc{i}_peak_distance_std'] = np.std(np.diff(peaks))
        else:
            features[f'mfcc{i}_peak_distance_mean'] = 0
            features[f'mfcc{i}_peak_distance_std'] = 0
        
        # Number of peaks normalized by length
        features[f'mfcc{i}_peak_density'] = len(peaks) / len(coeff)
    
    # 3. Spectral Features
    for i in range(20):
        coeff = mfcc_matrix[i, :]
        
        # FFT for frequency analysis
        fft_vals = np.abs(fft(coeff))
        
        # Spectral centroid (brightness of sound)
        freqs = np.fft.fftfreq(len(coeff))
        features[f'mfcc{i}_spectral_centroid'] = np.sum(freqs * fft_vals) / np.sum(fft_vals)
        
        # Spectral rolloff (shape of spectrum)
        cumsum = np.cumsum(fft_vals)
        rolloff_point = np.where(cumsum >= 0.85 * cumsum[-1])[0][0]
        features[f'mfcc{i}_spectral_rolloff'] = rolloff_point / len(fft_vals)
    
    # 4. Temporal Segmentation Features
    segment_size = mfcc_matrix.shape[1] // 3  # Split into three segments
    
    for i in range(20):
        coeff = mfcc_matrix[i, :]
        
        # Features for beginning, middle, and end of song
        features[f'mfcc{i}_begin_mean'] = np.mean(coeff[:segment_size])
        features[f'mfcc{i}_middle_mean'] = np.mean(coeff[segment_size:2*segment_size])
        features[f'mfcc{i}_end_mean'] = np.mean(coeff[2*segment_size:])
    
    # 5. Cross-MFCC Dynamic Features
    # Compute pairwise differences between consecutive frames
    deltas = np.diff(mfcc_matrix, axis=1)
    
    # Overall dynamics
    features['total_dynamics'] = np.mean(np.abs(deltas))
    features['dynamics_std'] = np.std(deltas)
    
    # Compute acceleration (second-order differences)
    accel = np.diff(deltas, axis=1)
    features['total_acceleration'] = np.mean(np.abs(accel))
    features['acceleration_std'] = np.std(accel)
    
    # 6. Structural Features
    # Silence detection (low energy frames)
    energy = np.sum(mfcc_matrix ** 2, axis=0)
    silence_threshold = np.mean(energy) * 0.1
    silence_frames = np.sum(energy < silence_threshold)
    features['silence_ratio'] = silence_frames / len(energy)
    
    # Variation over time windows
    window_size = min(100, mfcc_matrix.shape[1] // 10)
    for i in range(0, 20, 4):  # Take every 4th coefficient to reduce dimensionality
        coeff = mfcc_matrix[i, :]
        windows = np.array_split(coeff, 10)  # Split into 10 windows
        window_means = [np.mean(w) for w in windows]
        features[f'mfcc{i}_temporal_variation'] = np.std(window_means)
    
    return features

def get_feature_importance(X, y, n_estimators=100):
    """
    Train a Random Forest classifier and return feature importance
    """
    from sklearn.ensemble import RandomForestClassifier
    
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf.fit(X, y)
    
    # Get feature importance
    importance_dict = dict(zip(X.columns, rf.feature_importances_))
    return dict(sorted(importance_dict.items(), key=lambda x: x[1], reverse=True))

def visualize_features(features_df, target_labels=None):
    """
    Create visualizations of the features
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Select top numerical columns (exclude file_name and other non-numeric)
    numeric_cols = features_df.select_dtypes(include=[np.number]).columns
    
    # Create correlation heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(features_df[numeric_cols].corr(), cmap='coolwarm', center=0)
    plt.title('Feature Correlation Heatmap')
    plt.tight_layout()
    plt.show()
    
    if target_labels is not None:
        # Box plots for top features by class
        for col in numeric_cols[:5]:  # Plot top 5 features
            plt.figure(figsize=(10, 6))
            sns.boxplot(x=target_labels, y=features_df[col])
            plt.title(f'{col} Distribution by Class')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()



In [4]:
file_names = glob.glob('*.csv')
features_df = pd.DataFrame([extract_enhanced_mfcc_features(pd.read_csv(f, header=None).values) 
                          for f in file_names])

scaler = StandardScaler()
data_standardized = scaler.fit_transform(features_df)
data_standardized_df = pd.DataFrame(data_standardized, columns=features_df.columns)
data_standardized_df.to_csv("train_data_features.csv")